In [ ]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
import pandas as pd

In [10]:
demand = pd.read_csv('demand.csv')
machine_dtls = pd.read_csv('machine_dtls.csv')
capacity = pd.read_csv('pathway_compliments.csv')
sector_compliments = pd.read_csv('sector_compliments.csv')

In [11]:
capacity.Adj_Capacity_Daily_Per_Machine = capacity.Adj_Capacity_Daily_Per_Machine.replace(',', '', regex=True)
capacity.Adj_Capacity_Daily_Per_Machine = pd.to_numeric(capacity.Adj_Capacity_Daily_Per_Machine)

In [12]:
sector_compliments['Inverse_Compliment'] = 1/sector_compliments['Compliment']
demand_pw_exists = demand[~demand['PW'].isnull()].reset_index(drop=True)
machine_exists = machine_dtls[~machine_dtls['Work_Sector'].isnull()].reset_index(drop=True)

capacity_needed = capacity[capacity['Pathway'].isin(demand_pw_exists["PW"].unique())]
capacity_needed = capacity_needed[~capacity_needed['Work_Sector'].isnull()].reset_index(drop=True)

In [13]:
bot = []
for p in demand_pw_exists.PW.unique():
    bot.append(capacity_needed[capacity_needed['Pathway'] == p]['Adj_Capacity_Daily_Per_Machine'].min() * 90)
demand_pw_exists['Overall_Bottleneck'] = bot

In [14]:
demand_pw_exists

,BPC,PW,Past_Due_and_Current,Thirty,Sixty,Ninety,Overall_Bottleneck
0,158.0,CN58,7903,3072,3194,4013,32760.0
1,200.0,FG01,6340,319,955,1219,38999.7
2,202.0,FG02,7400,200,700,2057,292500.0
3,203.0,FG03,7474,15069,0,62966,719550.0
4,204.0,FG04,33681,5150,500,900,479700.0
5,206.0,FG05,26551,1126,300,0,175500.0
6,250.0,FG51,9913,228,546,1446,38999.7
7,254.0,FG52,6376,500,900,0,193050.0
8,303.0,TR01,21098,14050,59074,38850,719550.0
9,304.0,TR02,10740,7600,3551,4500,479700.0


,BPC,PW,Past_Due_and_Current,Thirty,Sixty,Ninety,Overall_Bottleneck
0,158.0,CN58,7903,3072,3194,4013,32760.0
1,200.0,FG01,6340,319,955,1219,38999.7
2,202.0,FG02,7400,200,700,2057,292500.0
3,203.0,FG03,7474,15069,0,62966,719550.0
4,204.0,FG04,33681,5150,500,900,479700.0
5,206.0,FG05,26551,1126,300,0,175500.0
6,250.0,FG51,9913,228,546,1446,38999.7
7,254.0,FG52,6376,500,900,0,193050.0
8,303.0,TR01,21098,14050,59074,38850,719550.0
9,304.0,TR02,10740,7600,3551,4500,479700.0


In [15]:
# choose which demand subset
demand_col = 'Overall_Bottleneck'

In [16]:
# Define model and parameters. 
mod1 = Model()

def flatten(t):
    return [item for sublist in t for item in sublist]

sectors = machine_dtls[~machine_dtls['Work_Sector'].isna()]['Work_Sector'].unique()
pathways = capacity_needed["Pathway"].unique()

# Variable Identifiers
days = range(90)
machine_count = range(len(machine_exists))
pw_count = range(len(pathways))
sectors_num = range(int(sectors.max()))
max_operations = range(capacity_needed['Operation_Sequence'].max())

# Machine Dictionary
machine_dic = {}

for i in machine_count:
    machine_dic[i] = machine_exists.iloc[i,0]

# Set pw to numbers
pw_indexing_dic = {}

for p in pw_count:
    pw_indexing_dic[p] = pathways[p]

# PW Dictionary
pw_dic = {} 

for p in pw_count:
    pw_dic[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
        pw_dic[p][o] = list(capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Work_Sector'])

# Capacity Dictionary
capacity_dic = {} 

for p in pw_count:
    capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]

# pathway demand
demand_dic = {} 

for p in pw_count:
    demand_dic[p] = demand_pw_exists[demand_pw_exists['PW'] == pathways[p]][demand_col].iloc[0]

# machines in sector
sector_dic = {}

for s in sectors_num:
    sector_dic[s] = list(machine_exists[machine_exists['Work_Sector'] == s+1].index)

# sector for each machine
ms_dic = {}

for m in machine_count:
    ms_dic[m] = machine_exists[machine_exists['Machine_Code'] == machine_dic[m]]['Work_Sector'].iloc[0]

# create previous capacity dataframe and previous capacity dic, bottleneck dataframe
prev_cap = np.zeros(len(capacity_needed))
count = 0
for p in pathways:
    for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence']:
        if o == 1:
            prev_cap[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]
            count += 1
            continue

        prev_cap[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o-1)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]
        count += 1

bottleneck = np.zeros(len(capacity_needed))
count = 0
next_count = 0
for p in pathways:
    next_count = count + len(capacity_needed[capacity_needed['Pathway'] == p])
    bottleneck[count:next_count] = capacity_needed[capacity_needed['Pathway'] == p]['Adj_Capacity_Daily_Per_Machine'].min()
    count = next_count

capacity_needed['Inverse_Capacity'] = 1/capacity_needed['Adj_Capacity_Daily_Per_Machine']

# Capacity Dictionary
inverse_capacity_dic = {} 

for p in pw_count:
    inverse_capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        inverse_capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Inverse_Capacity'].iloc[0]

capacity_needed['Adj_Capacity_Daily_Per_Machine_Last'] = prev_cap
capacity_needed['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'] = [min(capacity_needed['Adj_Capacity_Daily_Per_Machine_Last'].iloc[i], capacity_needed['Adj_Capacity_Daily_Per_Machine'].iloc[i]) for i in range(len(capacity_needed))]

sticky_min = np.zeros(len(capacity_needed))
sticky_op = np.zeros(len(capacity_needed))
count = 0
for p in pathways:
    for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence']:
        if o == 1:
            sticky_min[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]
            sticky_op[count] = o
            count += 1
            continue

        sticky_min[count] = min(sticky_min[count-1], capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0])

        if sticky_min[count-1] <= capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]:
            sticky_op[count] = sticky_op[count-1]
        else:
            sticky_op[count] = o

        count += 1

capacity_needed['Bottleneck'] = sticky_min
capacity_needed['Bottleneck_Op'] = sticky_op
capacity_needed['Overall_Bottleneck'] = bottleneck

op_cap_dic = {}

for p in pw_count:
    op_cap_dic[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
        op_cap_dic[p][o] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Bottleneck_Op'].iloc[0]

bottle_capacity_dic = {} 

for p in pw_count:
    bottle_capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        bottle_capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]

pw_op_kind_mach = {}

for p in pw_count:
    pw_op_kind_mach[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence']:
        pw_op_kind_mach[p][o] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Operation_Desc'].iloc[0]

op_kind_mach = {}
for i in capacity_needed['Operation_Desc'].unique():
    op_kind_mach[i] = [n for n in machine_exists[machine_exists['Machine_Type'] == i].index]

In [17]:
# Decision variables
employees = mod1.addVars(len(sectors_num), len(days), vtype = GRB.INTEGER)
machines = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations), vtype = GRB.BINARY)
production = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
#aux_prod = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
#possible_production = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
sum_emp_per_day = mod1.addVars(len(days))
max_sum_emp = mod1.addVars(1)
min_sum_ratio = mod1.addVars(1)

# Constraints

# there needs to be enough employees running the machines in each sector
machines_in_sectors_dic = {}
for s in sectors_num:
    if len(sector_dic[s]) == 0:
        continue
    machines_in_sectors_dic[s] = {}
    for d in days:
        machines_in_sectors_dic[s][d] = mod1.addConstr(sum([machines[m,d,p,o] for m in sector_dic[s] for p in pw_count for o in max_operations]) * sector_compliments[sector_compliments['Work_Sector'] == s+1]['Inverse_Compliment'].iloc[0] <= employees[s,d])

#machines_in_sectors_dic = {}
#for s in sectors_num:
#    if len(sector_dic[s]) == 0:
#        continue
#    machines_in_sectors_dic[s] = {}
#    for d in days:
#        machines_in_sectors_dic[s][d] = mod1.addConstr(sum([production[m,d,p,o]*inverse_capacity_dic[p][s] for m in sector_dic[s] for p in pw_count for o in max_operations]) * sector_compliments[sector_compliments['Work_Sector'] == s+1]['Inverse_Compliment'].iloc[0] <= employees[s,d])        
        
# a machine can only serve one pathway_operation a day
single_pathway = {}
for m in machine_count:
    single_pathway[m] = {}
    for d in days:
        single_pathway[m][d] = mod1.addConstr(sum([machines[m,d,p,o] for p in pw_count for o in max_operations]) <= 1)

#single_pathway = {}
#for m in machine_count:
#    single_pathway[m] = {}
#    for d in days:
#        single_pathway[m][d] = mod1.addConstr(sum([production[m,d,p,o]*inverse_capacity_dic[p][ms_dic[m]] for p in pw_count for o in max_operations]) <= 1)
        
# order of production sequence must be maintained for daily machine assignemnt. An operation may not produce product if its prior operation has not produced at least the number it plans to produce that day.
# sequence = {}
# for p in pw_count:
#    sequence[p] = {}
#    for o in range(len(pw_dic[p].keys())-1):
#        sequence[p][o] = {}
#        for z in days:
#            sequence[p][o][z] = mod1.addConstr(sum([machines[m,d,p,o] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) - sum([machines[m,z,p,o] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= sum([machines[m,z,p,o+1] * bottle_capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

# order of production sequence must be maintained for daily machine assignemnt. An operation may not produce product if its prior operation has not produced at least the number it plans to produce that day.
#aux = {}
#for p in pw_count:
#    aux[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        aux[p][o] = {}
#        for z in days:
#            aux[p][o][z] = {}
#            for mk in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
#                aux[p][o][z][mk] = mod1.addConstr(aux_prod[mk,z,p,o] == sum([machines[m,d,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) - sum([machines[m,z,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]))

#poss_cap = {}
#for p in pw_count:
#    poss_cap[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        poss_cap[p][o] = {}
#        for z in days:
#            poss_cap[p][o][z] = {}
#            for mk in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
#                poss_cap[p][o][z][mk] = mod1.addConstr(possible_production[mk,z,p,o] == min_([aux_prod[mk,z,p,o], capacity_dic[p][ms_dic[mk]]]))

#sequence = {}
#for p in pw_count:
#    sequence[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        sequence[p][o] = {}
#        for z in days:
#            sequence[p][o][z] = mod1.addConstr(sum([machines[m,d,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) - sum([machines[m,z,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) >= sum([machines[m,z,p,o+1] * possible_production[m,z,p,o+1] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

sequence = {}
for p in pw_count:
    sequence[p] = {}
    for o in range(len(pw_dic[p].keys())-1):
        sequence[p][o] = {}
        for z in days:
            sequence[p][o][z] = mod1.addConstr(sum([production[m,d,p,o] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) - sum([production[m,z,p,o] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= sum([production[m,d,p,o+1] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

# machine can't produce more than its capacity

capacity = {}
for p in pw_count:
    capacity[p] = {}
    for o in range(len(pw_dic[p].keys())-1):
        capacity[p][o] = {}
        for d in days:
            capacity[p][o][d] = {}
            for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
                capacity[p][o][d][m] = mod1.addConstr(production[m,d,p,o] <= capacity_dic[p][ms_dic[m]])
            
# turn off extraneous machines that cant produce output due to insufficient wip (other minimization efforts may cause this by default, but this is for extra saftey)
zero_default = {}
for p in pw_count:
    zero_default[p] = {}
    for o in max_operations:
        zero_default[p][o] = {}
        for z in days:
            zero_default[p][o][z] = {}
            for m in machine_count:
                zero_default[p][o][z][m] = mod1.addConstr(0 <= production[m,z,p,o])
                
zero_default2 = {}
for p in pw_count:
    zero_default2[p] = {}
    for o in max_operations:
        zero_default2[p][o] = {}
        for z in days:
            zero_default2[p][o][z] = {}
            for m in machine_count:
                zero_default2[p][o][z][m] = mod1.addConstr(machines[m,z,p,o] * production[m,z,p,o] == production[m,z,p,o])

# the number of machines running needs to produce enough output for each pathway they support
# output = {}
# for p in pw_count:
#    output[p] = {}
#    for o in range(len(pw_dic[p].keys())):
#        output[p][o] = mod1.addConstr(sum([machines[m,d,p,o] * possible_production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= demand_dic[p])

output = {}
for p in pw_count:
    output[p] = {}
    for o in range(len(pw_dic[p].keys())):
        output[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) <= demand_dic[p])        

complete = {}
for p in pw_count:
    complete[p] = {}
    for o in range(len(pw_dic[p].keys())-1):
        complete[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) == sum([production[m,d,p,o+1] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))        
        
# only proper sectors can support a certain operations
reduce_sector = {}
for p in pw_count:
    reduce_sector[p] = {}
    for o in range(len(pw_dic[p].keys())):
        reduce_sector[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in [i for i in machine_count if i not in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]) == 0)

# only proper machines in sectors can support certain operations
#reduce_machine = {}
#for p in pw_count:
#    reduce_machine[p] = {}
#    for o in range(len(pw_dic[p].keys())):
#        reduce_machine[p][o] = mod1.addConstr(sum([machines[m,d,p,o] for d in days for m in [i for i in machine_count if i not in [g for g in op_kind_mach[pw_op_kind_mach[p][o+1]] if g in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]]) == 0)

reduce_product = {}
for p in pw_count:
    reduce_product[p] = {}
    for o in range(len(pw_dic[p].keys())):
        reduce_product[p][o] = {}
        for d in days:
            reduce_product[p][o][d] = {}
            for m in [i for i in machine_count if i not in [g for g in op_kind_mach[pw_op_kind_mach[p][o+1]] if g in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]:
                reduce_product[p][o][d][m] = mod1.addConstr(production[m,d,p,o] == 0)

reduce_machine = {}
for p in pw_count:
    reduce_machine[p] = {}
    temp = [x for x in max_operations if x not in [i for i in range(len(pw_dic[p].keys()))]]
    for o in temp:
        reduce_machine[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in machine_count]) == 0)     
    

# set only two machines to a pathway per day
two_pathway = {}
for p in pw_count:
    two_pathway[p] = {}
    for d in days:
        two_pathway[p][d] = {}
        for o in range(len(pw_dic[p].keys())):
            two_pathway[p][d][o] = mod1.addConstr(sum([machines[m,d,p,o] for m in machine_count]) <= 2)
        
# number of employees used each day needs to be the sum of the employees in all sectors
emp_per_day_dic = {}
for d in days:
    emp_per_day_dic[d] = mod1.addConstr(sum([employees[s,d] for s in sectors_num]) == sum_emp_per_day[d])

m_ax = {}
for d in days:
    m_ax[d] = mod1.addConstr(max_sum_emp[0] >= sum_emp_per_day[d])

m_ax_ratio = {}
for p in pw_count:
    m_ax_ratio[p] = {}
    for o in range(len(pw_dic[p].keys())):
        m_ax_ratio[p][o] = mod1.addConstr(min_sum_ratio[0] >= -1*((sum([production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])])/demand_dic[p])-1))

# Set objective function
mod1.setObjective(min_sum_ratio[0], GRB.MINIMIZE)


In [18]:
# Update and solve
mod1.setParam(GRB.Param.Method, 3)
mod1.setParam(GRB.Param.NodeMethod, 2)
mod1.setParam(GRB.Param.Presolve, 2)
mod1.setParam(GRB.Param.MIPFocus, 1)
mod1.update()
mod1.optimize()

Changed value of parameter Method to 3
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Method to 3
Changed value of parameter NodeMethod to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter NodeMethod to 2
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 2
Changed value of parameter MIPFocus to 1
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
   Prev: 0  Min: 0  Max: 3  Default: 0
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Optimize a model with 15675762 rows, 19053002 columns and 68472621 nonzeros
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Opti

Presolve removed 15615920 rows and 18687270 columns (presolve time = 236s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 240s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 240s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 245s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 245s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 250s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 250s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 255s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 255s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 260s) ...
Presolve removed 15615920 rows and 18687270 columns (presolve time = 260s) ...
Presolve removed 15616087 rows and 18688377 columns (presolve time = 266s) ...
Presolve removed 15616087 rows and 18688377 columns 

  22   3.75446546e-01  2.86723763e-01  1.47e-01 1.22e-10  2.10e-07  1217s

Barrier performed 22 iterations in 1218.80 seconds

Barrier solve interrupted - model solved by another algorithm
Barrier performed 22 iterations in 1218.80 seconds

Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex

Solved with dual simplex


Root relaxation: objective 3.727709e-01, 34219 iterations, 198.98 seconds
Root relaxation: objective 3.727709e-01, 34219 iterations, 198.98 seconds
Total elapsed time = 1219.39s
Total elapsed time = 1219.39s
Total elapsed time = 1348.00s
Total elapsed time = 1348.00s
Total elapsed time = 1355.37s
Total elapsed time = 1355.37s
Total elapsed time = 1364.64s
Total elapsed time = 1364.64s

    Nodes    |    Current Node    |     Objective Bounds      |     Work

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time
    Nodes    |    Current Node    |     Objective Bounds      |     Work

 Expl Unexpl |  Obj  Depth I

In [19]:
mod1.objval

0.42222222222222217

0.42222222222222217

In [20]:
machine_num = [i[0] for i in production if production[i].x >= 1]
solution_machines = [machine_dic[i[0]] for i in production if production[i].x >= 1]
soltion_days = [i[1]+1 for i in production if production[i].x >= 1]
solution_pw = [pathways[i[2]] for i in production if production[i].x >= 1]
solution_operation = [i[3]+1 for i in production if production[i].x >= 1]
solution_desc = [pw_op_kind_mach[i[2]][i[3]+1] for i in production if production[i].x >= 1]
solution_sector = [ms_dic[i[0]] for i in production if production[i].x >= 1]
solution_produced = [production[i].x for i in production if production[i].x >= 1]

In [21]:
pd.options.display.max_rows = 999
active = pd.DataFrame({'Machine_Num': machine_num, 'Machine': solution_machines, 'Day': soltion_days, 'PW': solution_pw, 'Operation': solution_operation, 'Description': solution_desc, 'Sector': solution_sector, 'Produced': solution_produced}).sort_values(by=['Day'])

work_array = np.zeros([len(sectors_num), len(days)])

for s in sectors_num:
    work_array[s,:] = [employees[s,d].x for d in days]
    
emp_schedule = pd.DataFrame(work_array, index = [f'Sector {i+1}' for i in sectors_num], columns = [f'Day {i+1}' for i in days])
emp_schedule = emp_schedule.round()

labor_mix_dic = {}

for d in days:
    labor_mix_dic[d+1] = {}
    if len(active[active['Day'] == d+1]['Sector'].unique()) == 0:
        continue
    for s in active[active['Day'] == d+1]['Sector'].unique():
        if len(active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].unique()) > 1:
            for desc in active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].unique():
                labor_mix_dic[d+1][desc] = len(active[(active['Day'] == d+1) & (active['Sector'] == s) & (active['Description'] == desc)])
        else:
            labor_mix_dic[d+1][active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].iloc[0]] = work_array[int(s)-1,d]

In [22]:
labor_req = pd.DataFrame(labor_mix_dic).fillna(0)
labor_req['Max'] = labor_req.max(axis = 1)
labor_req.loc['Sum'] = labor_req.sum()
labor_req

,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,Max
Header,16.0,15.0,15.0,13.0,13.0,13.0,14.0,12.0,11.0,13.0,...,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
Heat Treat,0.0,2.0,2.0,4.0,6.0,5.0,6.0,5.0,6.0,4.0,...,7.0,7.0,6.0,4.0,5.0,5.0,0.0,0.0,0.0,7.0
Pointer,0.0,1.0,3.0,3.0,3.0,4.0,3.0,3.0,2.0,2.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
Pointer-Lathe,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Trimmer,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
CNC,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0
Stamper,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
Grinder,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,3.0,...,2.0,1.0,2.0,1.0,2.0,2.0,3.0,0.0,0.0,8.0
Fillet Roller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,...,1.0,2.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,6.0
Slotter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,Max
Header,16.0,15.0,15.0,13.0,13.0,13.0,14.0,12.0,11.0,13.0,...,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
Heat Treat,0.0,2.0,2.0,4.0,6.0,5.0,6.0,5.0,6.0,4.0,...,7.0,7.0,6.0,4.0,5.0,5.0,0.0,0.0,0.0,7.0
Pointer,0.0,1.0,3.0,3.0,3.0,4.0,3.0,3.0,2.0,2.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
Pointer-Lathe,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Trimmer,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
CNC,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0
Stamper,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
Grinder,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,3.0,...,2.0,1.0,2.0,1.0,2.0,2.0,3.0,0.0,0.0,8.0
Fillet Roller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,...,1.0,2.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,6.0
Slotter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
active

,Machine_Num,Machine,Day,PW,Operation,Description,Sector,Produced
972,43,HH10,1,TR01,1,Header,1.0,23569.0
1123,52,HH52,1,TR56,1,Header,1.0,4095.0
1092,50,HH47,1,TR56,1,Header,1.0,4095.0
825,38,H14,1,TR03,1,Header,1.0,7150.0
1142,53,HH7,1,TR51,1,Header,1.0,16250.0
...,...,...,...,...,...,...,...,...
5151,208,TR76,90,TR02,7,Thread Roller,43.0,277160.0
2307,100,R50,90,CN04,8,Thread Roller,27.0,46800.0
816,32,FG100,90,FG03,11,Flute Grinder,47.0,78000.0
2301,96,R3,90,TR52,7,Thread Roller,8.0,10010.0


,Machine_Num,Machine,Day,PW,Operation,Description,Sector,Produced
972,43,HH10,1,TR01,1,Header,1.0,23569.0
1123,52,HH52,1,TR56,1,Header,1.0,4095.0
1092,50,HH47,1,TR56,1,Header,1.0,4095.0
825,38,H14,1,TR03,1,Header,1.0,7150.0
1142,53,HH7,1,TR51,1,Header,1.0,16250.0
...,...,...,...,...,...,...,...,...
5151,208,TR76,90,TR02,7,Thread Roller,43.0,277160.0
2307,100,R50,90,CN04,8,Thread Roller,27.0,46800.0
816,32,FG100,90,FG03,11,Flute Grinder,47.0,78000.0
2301,96,R3,90,TR52,7,Thread Roller,8.0,10010.0


In [24]:
emp_schedule

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10,...,Day 81,Day 82,Day 83,Day 84,Day 85,Day 86,Day 87,Day 88,Day 89,Day 90
Sector 1,16.0,15.0,15.0,13.0,13.0,13.0,14.0,12.0,11.0,13.0,...,9.0,11.0,10.0,6.0,2.0,3.0,0.0,1.0,0.0,1.0
Sector 2,-0.0,-0.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,1.0,2.0,1.0,2.0,-0.0,1.0,-0.0,-0.0
Sector 3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0
Sector 4,-0.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,3.0,...,2.0,4.0,5.0,3.0,3.0,3.0,1.0,1.0,1.0,-0.0
Sector 5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,2.0,3.0,2.0,3.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,1.0,-0.0
Sector 6,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,...,4.0,2.0,1.0,3.0,3.0,3.0,4.0,2.0,0.0,0.0
Sector 7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0
Sector 8,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,...,2.0,3.0,2.0,1.0,1.0,2.0,2.0,3.0,4.0,5.0
Sector 9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Sector 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10,...,Day 81,Day 82,Day 83,Day 84,Day 85,Day 86,Day 87,Day 88,Day 89,Day 90
Sector 1,16.0,15.0,15.0,13.0,13.0,13.0,14.0,12.0,11.0,13.0,...,9.0,11.0,10.0,6.0,2.0,3.0,0.0,1.0,0.0,1.0
Sector 2,-0.0,-0.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,1.0,2.0,1.0,2.0,-0.0,1.0,-0.0,-0.0
Sector 3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0
Sector 4,-0.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,3.0,...,2.0,4.0,5.0,3.0,3.0,3.0,1.0,1.0,1.0,-0.0
Sector 5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,2.0,3.0,2.0,3.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,1.0,-0.0
Sector 6,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,...,4.0,2.0,1.0,3.0,3.0,3.0,4.0,2.0,0.0,0.0
Sector 7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0
Sector 8,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,...,2.0,3.0,2.0,1.0,1.0,2.0,2.0,3.0,4.0,5.0
Sector 9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Sector 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
sum_prod_dic = {}
for p in pathways:
    sum_prod_dic[p] = {}
    for d in days:
        sum_prod_dic[p][d+1] = {}
        for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence'].unique():
            sum_prod_dic[p][d+1][o] = 0


for p in pw_count:
    for d in days:
        for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
            if len(active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]) == 0:
                if d == 0:
                    continue
                else:
                    sum_prod_dic[pathways[p]][d+1][o] = sum_prod_dic[pathways[p]][d][o]
            else:
                if d == 0:
                    sum_prod_dic[pathways[p]][d+1][o] += sum([production[m,d,p,o-1].x for m in active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]['Machine_Num']])
                else:
                    sum_prod_dic[pathways[p]][d+1][o] += sum([production[m,d,p,o-1].x for m in active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]['Machine_Num']]) + sum_prod_dic[pathways[p]][d][o]

In [26]:
pd.DataFrame(sum_prod_dic['CN58'])

,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
1,0,0,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
2,0,0,0.00,728.00,1092.00,1092.00,1755.00,2483.00,2483.00,2483.00,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
3,0,0,0.00,0.00,728.00,728.00,728.00,728.00,1605.50,1605.50,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
4,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,17335.49,17335.49,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
5,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,15645.50,16208.83,17335.49,17335.49,18462.15,19025.48,19025.48,19025.48,19025.48,19025.48
6,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,14345.48,14345.48,14345.48,14345.48,14345.48,17465.48,19025.48,19025.48,19025.48,19025.48
7,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,12070.48,12070.48,12070.48,14345.48,14345.48,14345.48,16620.48,18895.48,19025.48,19025.48
8,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19025.48


,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
1,0,0,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,16113.48,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
2,0,0,0.00,728.00,1092.00,1092.00,1755.00,2483.00,2483.00,2483.00,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
3,0,0,0.00,0.00,728.00,728.00,728.00,728.00,1605.50,1605.50,...,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
4,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,17335.49,17335.49,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48,19025.48
5,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,15645.50,16208.83,17335.49,17335.49,18462.15,19025.48,19025.48,19025.48,19025.48,19025.48
6,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,14345.48,14345.48,14345.48,14345.48,14345.48,17465.48,19025.48,19025.48,19025.48,19025.48
7,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,12070.48,12070.48,12070.48,14345.48,14345.48,14345.48,16620.48,18895.48,19025.48,19025.48
8,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19025.48


In [37]:
pd.options.display.max_columns = 999

with pd.ExcelWriter('Bottleneck.xlsx') as writer:  

    pd.DataFrame(demand_pw_exists[['PW','Overall_Bottleneck']]).to_excel(writer, sheet_name = 'Demand')
    
    labor_req.to_excel(writer, sheet_name = 'Labor_Requirement')

    for p in pathways:
        active[active['PW'] == p].to_excel(writer, sheet_name = p)

        pd.DataFrame(sum_prod_dic[p]).to_excel(writer, sheet_name = p, startcol = len(active[active['PW'] == p].columns)+2)

,PW,Overall_Bottleneck
0,CN58,32760.0
1,FG01,38999.7
2,FG02,292500.0
3,FG03,719550.0
4,FG04,479700.0
5,FG05,175500.0
6,FG51,38999.7
7,FG52,193050.0
8,TR01,719550.0
9,TR02,479700.0


,PW,Overall_Bottleneck
0,CN58,32760.0
1,FG01,38999.7
2,FG02,292500.0
3,FG03,719550.0
4,FG04,479700.0
5,FG05,175500.0
6,FG51,38999.7
7,FG52,193050.0
8,TR01,719550.0
9,TR02,479700.0
